In [2]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import warnings

# Suppress future warnings (from pandas/plotly)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the dataset
df = pd.read_csv("student_performance.csv")

# Initialize the app with Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App Layout
app.layout = dbc.Container([
    html.H1("📊 Student Performance Dashboard", className="text-center text-primary mb-4"),

    # KPI Cards Row
    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("Average Attendance"),
            dbc.CardBody(html.H4(f"{df['Attendance (%)'].mean():.2f}%", className="card-title text-success"))
        ], color="light"), width=4),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Top Performing Grade"),
            dbc.CardBody(html.H4(df.groupby("Grade")["Math Score"].mean().idxmax(), className="card-title text-info"))
        ], color="light"), width=4),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Average Score (All Subjects)"),
            dbc.CardBody(html.H4(f"{df[['Math Score', 'Science Score', 'English Score', 'History Score', 'Computer Science Score']].mean().mean():.2f}", className="card-title text-warning"))
        ], color="light"), width=4),
    ], className="mb-4"),

    # Filters Row
    dbc.Row([
        dbc.Col([
            html.Label("Filter by Grade"),
            dcc.Dropdown(
                options=[{"label": grade, "value": grade} for grade in sorted(df["Grade"].unique())],
                id="grade-filter",
                value=None,
                placeholder="Select Grade",
                clearable=True
            )
        ], width=4),

        dbc.Col([
            html.Label("Select Subject"),
            dcc.Dropdown(
                options=[{"label": s, "value": s} for s in [
                    "Math Score", "Science Score", "English Score",
                    "History Score", "Computer Science Score"
                ]],
                id="subject-dropdown",
                value="Math Score"
            )
        ], width=4),
    ], className="mb-4"),

    # Graphs Row
    dbc.Row([
        dbc.Col(dcc.Graph(id="subject-bar-chart"), width=6),
        dbc.Col(dcc.Graph(id="attendance-scatter"), width=6)
    ])
], fluid=True)

# Callback to update both charts
@app.callback(
    [Output("subject-bar-chart", "figure"),
     Output("attendance-scatter", "figure")],
    [Input("grade-filter", "value"),
     Input("subject-dropdown", "value")]
)
def update_charts(selected_grade, subject):
    filtered_df = df[df["Grade"] == selected_grade] if selected_grade else df

    # Bar Chart: Subject Score by Student
    bar_fig = px.bar(
        filtered_df, x="Student Name", y=subject, color="Grade",
        title=f"{subject} by Student", labels={subject: 'Score'}
    )

    # Scatter Plot: Attendance vs Subject Score
    scatter_fig = px.scatter(
        filtered_df, x="Attendance (%)", y=subject, color="Grade", size=subject,
        title=f"Attendance vs {subject}", labels={"Attendance (%)": "Attendance (%)", subject: "Score"}
    )

    return bar_fig, scatter_fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
